In [15]:
import pandas as pd

In [16]:
def string_to_set(string):
    res = set(string.replace('{', '').replace('}', '').replace('\\', '').replace("'", '').replace('"', '').strip().split(','))
    clear_res = set()
    for elem in res:
        clear_res.add(elem.strip())
    return clear_res

def get_all_lemmas(lemmas_column):
    lemmas = set()
    for row in lemmas_column:
        lemmas = lemmas.union(string_to_set(row))

    return lemmas


df1 = pd.read_csv('l2_book.csv')
books = get_all_lemmas(df1[~df1.lemmas.isna()]['lemmas'])

df2 = pd.read_csv('l2_film.csv')
films = get_all_lemmas(df2[~df2.lemmas.isna()]['lemmas'])

In [17]:
def get_lemmas_set():
    df = pd.read_csv('genres_clear.csv')
    return set(df['genre'].values)

lemmas = get_lemmas_set()

In [18]:
df1['lemmas'] = df1['lemmas'].apply(lambda x: string_to_set(x)) 
df2['lemmas'] = df2['lemmas'].apply(lambda x: string_to_set(x))

In [19]:
df1['lemmas_inter'] = df1['lemmas'].apply(lambda x: x.intersection(lemmas)) 
df2['lemmas_inter'] = df2['lemmas'].apply(lambda x: x.intersection(lemmas)) 

In [29]:
def reccomend(title: str, genres_col='lemmas_inter'):
    id_ = df1[df1.title == title].head(1).id.item()
    print(id_)
    item = (df1.at[id_, genres_col])
    print(item)
    df2['rec'] = df2[genres_col].apply(
                        lambda x: len((x).intersection(item))/(len(item) + 2 * len(x))
                                        )
    return df2.sort_values(['rec', 'popularity', 'year'], ascending=False).head(5)

In [30]:
reccomend('Преступление и наказание')

7847
{'роман', 'вестник', 'петербург', 'журнал', 'литературный', 'русский', 'литература', 'классика', 'произведение'}


,Unnamed: 0,id,title,genres,year,description,content_type,popularity,lemmas,lemmas_inter,rec
12168,12168,12168,Преступление и наказание,драмы,1969.0,Бедный студент Родион Раскольников идет на пре...,series,0.450936,"{, роман, опубликовать, год, вестник, драма, п...","{роман, вестник, драма, петербург, журнал, лит...",0.280000
4190,4190,4190,Преступление и наказание,"драмы, русские",1969.0,"Родион Раскольников пытается понять, к какому ...",series,0.291904,"{, роман, опубликовать, год, русские, вестник,...","{роман, вестник, драма, петербург, журнал, лит...",0.280000
21165,21165,21165,Преступление и наказание,"русские, экранизации, драмы, советские, мирова...",1969.0,Экранизация одного из главных романов русской ...,series,0.352529,"{, опубликовать, русские, петербург, журнал, ф...","{роман, драма, вестник, петербург, журнал, лит...",0.258065
20001,20001,20001,Братья Карамазовы,драмы,2008.0,XIX век. В уездном российском городе разгорают...,series,0.575510,"{, роман, опубликовать, год, драма, вестник, п...","{роман, драма, вестник, психологический, журна...",0.240000
20356,20356,20356,Отцы и дети,"драмы, исторические, советские",1959.0,Советская экранизация одноименного знаменитого...,film,0.305952,"{, роман, опубликовать, год, драма, язык, вест...","{роман, драма, вестник, журнал, литературный, ...",0.222222


In [9]:
df1.drop('Unnamed: 0', axis=1).to_csv('books_with_lemmas.csv')

In [10]:
df2.drop('Unnamed: 0', axis=1).to_csv('films_with_lemmas.csv')

In [18]:
pd.set_option('display.max_colwidth', 200)
display(df1[['title', 'lemmas', 'lemmas_inter']])

,title,lemmas,lemmas_inter
0,Как выдать ведьму замуж,"{академия, фэнтези, юмористический, магический, любовный}","{юмористический, фэнтези, любовный}"
1,На острие удара,"{фантастика, боевой, героический}","{фантастика, боевой}"
2,Другой мир. Кровавое дерево. Книга 5,"{героический, фэнтези, боевой, приключение, книга, исторический}","{исторический, фэнтези, приключение, боевой}"
3,Магазин с привидением,"{приключение, вампир, детский, книга, зарубежный}","{детский, вампир, приключение, зарубежный}"
4,Один дома 6,"{оригинальный, фантастика, семейный, disney+, century, английский, космический, 20th, алфавит, фильм, производство, пандемия, приостановить, перезапуск, сша, год, studios, covid-19, язык}","{сша, английский, оригинальный, covid-19, фантастика, пандемия, семейный, космический, перезапуск, 20th}"
...,...,...,...
59017,Необыкновенная история,"{ребёнок, дошкольник, книга, сказка}","{ребёнок, сказка}"
59018,Ллойс,"{фантастика, боевой}","{фантастика, боевой}"
59019,Иуда Искариот,"{литература, список, повесть, классика, леонид, век, русский, алфавит, класс, андреев, книга, школьный}","{повесть, литература, русский, классика}"
59020,Любишь ли ты меня?,"{литература, подросток, русский, современный, книга}","{литература, русский}"


In [ ]:
df2[]